In [ ]:
def get_3body_angles(COORDS, hist_bins):
    theta = np.zeros((nFrames, nDrops))
    theta_hist = np.zeros((nFrames, len(hist_bins)-1))

    for frame in tqdm(range(nFrames)):
        coords = COORDS[frame*nDrops:(frame+1)*nDrops,:]
        kd = KDTree(coords)
        # find the 2 nearest neighbors, 3 since it counts itself
        _, n = kd.query(coords, 3)
        # remove self from neighbors
        n = np.array(n)[:, 1:]
        # displacement vectors
        r_j = coords[n[:, 0]] - coords
        r_k = coords[n[:, 1]] - coords
        # angle between the two vectors

        scalar_prod = np.zeros(nDrops)
        for i in range(nDrops):
            scalar_prod[i] = np.dot(r_j[i], r_k[i])

        theta[frame] = np.arccos(scalar_prod / (np.linalg.norm(r_j, axis=1) * np.linalg.norm(r_k, axis=1))) 
        theta_hist[frame], _ = np.histogram(theta[frame], bins=hist_bins, density=True)

    return theta, theta_hist

In [ ]:
hist_bins = np.linspace(0, np.pi, 10)
print("Calculating 3-body angles for raw trajs...")
theta, theta_hist = get_3body_angles(np.array(rawTrajs.loc[:,["x","y"]]), hist_bins)

print("Calculating 3-body angles for smooth trajs...")
theta_smooth, theta_hist_smooth = get_3body_angles(np.array(smoothTrajs.loc[:,["x","y"]]), hist_bins)

In [ ]:
print(theta - theta_smooth)

In [ ]:
if animated_plot_verb:
    fig, ax = plt.subplots(1, 1, figsize = (8, 5))
    anim_running = True

    def onClick(event):
        global anim_running
        if anim_running:
            ani.event_source.stop()
            anim_running = False
        else:
            ani.event_source.start()
            anim_running = True


    def prepare_animation(bar_container):
        def animate(frame):
            title.set_text(f"- Raw Trajectories - {frame/10} s")
            
            n, _ = np.histogram(theta[frame], hist_bins, density = True)
            for count, rect in zip(n, bar_container.patches):
                rect.set_height(count)
                
            return bar_container.patches
        return animate

    _, _, bar_container = ax.hist(theta[0], hist_bins, **default_kwargs_blue, label="blue particles")
    title = ax.set_title("- Raw Trajectories - 0 s")
    ax.set_xlabel(r"$\theta$")
    ax.set_ylabel("density")
    ax.set_ylim(0, 1)
    plt.tight_layout()
    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, prepare_animation(bar_container), range(0, nFrames, 100), repeat=False, blit=False)
    if save_verb:
        ani.save('./results/3body/3body_wind_raw.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    if show_verb:
        plt.show()
    else:
        plt.close()

    fig, ax = plt.subplots(1, 1, figsize = (8, 5))
    anim_running = True

    def prepare_animation(bar_container):
        def animate(frame):
            title.set_text(f"- Raw Trajectories - {frame/10} s")
            
            n, _ = np.histogram(theta_smooth[frame], hist_bins, density = True)
            for count, rect in zip(n, bar_container.patches):
                rect.set_height(count)
                
            return bar_container.patches
        return animate

    _, _, bar_container = ax.hist(theta_smooth[0], hist_bins, **default_kwargs_blue, label="blue particles")
    title = ax.set_title("- Raw Trajectories - 0 s")
    ax.set_xlabel(r"$\theta$")
    ax.set_ylabel("density")
    ax.set_ylim(0, 1)
    plt.tight_layout()
    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, prepare_animation(bar_container), range(0, nFrames, 100), repeat=False, blit=False)
    if save_verb:
        ani.save('./results/3body/3body_wind_smooth.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    if show_verb:
        plt.show()
    else:
        plt.close()

In [ ]:
X, Y = np.meshgrid(hist_bins[:-1] + np.diff(hist_bins) / 2, np.linspace(0, nFrames, nFrames))

fig, ax = plt.subplots(1, 1, figsize = (8, 6))
img = ax.pcolormesh(X, Y, theta_hist, cmap="viridis", shading="auto")
fig.colorbar(img)
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("Time [s]")
ax.set_xticks(hist_bins[:-1] + np.diff(hist_bins) / 2)
ax.set_xticklabels(np.round(hist_bins[:-1] + np.diff(hist_bins) / 2))
ax.set_aspect(1/8000)
plt.ylabel("Time [s]")
if save_verb: 
    plt.savefig("./results/3body/3body_wind_raw_hist.png", bbox_inches="tight")
if show_verb:
    plt.show()
else:
    plt.close()

fig, ax = plt.subplots(1, 1, figsize = (8, 6))
img = ax.pcolormesh(X, Y, theta_hist, cmap="viridis", shading="auto")
fig.colorbar(img)
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("Time [s]")
ax.set_xticks(hist_bins[:-1] + np.diff(hist_bins) / 2)
ax.set_xticklabels(np.round(hist_bins[:-1] + np.diff(hist_bins) / 2))
ax.set_aspect(1/8000)
plt.ylabel("Time [s]")
if save_verb: 
    plt.savefig("./results/3body/3body_wind_smooth_hist.png", bbox_inches="tight")
if show_verb:
    plt.show()
else:
    plt.close()